# OpenWeatherClient

In [3]:
import warnings
import json
import datetime
import zipfile
import bz2
import base64
import struct
import abc
import lzma
import io
import pickle
from types import SimpleNamespace
from pathlib import Path

import trio
import httpx
import pint
import pandas as pd
import matplotlib.pyplot as plt

u = pint.get_application_registry()

%autoawait trio
%matplotlib inline

In [20]:
def to_ns(obj):
    if isinstance(obj, dict):
        return SimpleNamespace(**{k:to_ns(v) for k,v in obj.items()})
    if isinstance(obj, (tuple, list)):
        return type(obj)(to_ns(v) for v in obj)
    return obj

In [22]:
# Handles HTTP-Requests in general (Parent-Class)

class RequestWrapper:
    
    # All Child-Classes must implement the Method standard_request
    @abc.abstractmethod
    async def standard_request(self, method, url, *args, **kw):
        pass
    
    # Method to get a request; all Child-Classes can use this Method
    async def get(self, *args, **kw):
        return await self.standard_request("get",*args,**kw)
    
    # Method to send a request; all Child-Classes can use this Method
    async def post(self, *args, **kw):
        return await self.standard_request("post",*args,**kw)

In [23]:
# Handles the OpenWeather-Request

class OpenWeatherClient(RequestWrapper):
    
    # Instantiate the class; it takes the parameters api_url and api_key
    def __init__(self, api_key:str, api_url:str = "https://api.openweathermap.org/data/3.0"):
        
        # Instantiate a new httpx-AsyncClient and tells server to send JSON-File back
        self._client = httpx.AsyncClient(headers = {"Accept": "application/json"})
        
        # Must-Have-Parameters
        self._api_url = api_url
        self._api_key = api_key

        
    async def __aenter__(self):
        await self._client.__aenter__()
        return self
    
    # When an exception happens inside the `with`-Block `__aexit__` will receive Information about the exception in its arguments.
    # The httpx-AsyncClient's `__aexit__`-Methode will be invoked and will decide if the exception is to propagete
    # (It always decides to propagate)
    # `__aexit__' closes the connection
    async def __aexit__(self, exc_type, exc_value, exc_tb):
        return await self._client.__aexit__(exc_type, exc_value, exc_tb)

    
    async def request(self, method, url, *args, **kw):
        full_url = f"{self._api_url}/{url}"
        response = await self._client.request(method, full_url, *args, **kw)
        return response
        
    async def standard_request(self, method, url, *args, **kw):
        params = kw.pop("params",{})
        assert params.setdefault("appid",self._api_key) == self._api_key
        response = await self.request(method,url, *args, params=params, **kw)
        response.raise_for_status()
        ret = to_ns(response.json()) # changes JSON to namespace-Object
        return ret

    
    # Defines the weatherstation with the coordinates as parameters
    def station_at(self,lat,lon):
        return WeatherStation(self,lat,lon)

In [33]:
# Handles a specific weatherstation

class WeatherStation(RequestWrapper):
    def __init__(self, client:OpenWeatherClient, lat:float, lon:float):
        self._client = client
        self._coordinates = (lat,lon)
    
    async def standard_request(self, method, url, *args, **kw):
        lat,lon = self._coordinates
        params = kw.pop("params",{})
        assert params.setdefault("lat",lat) == lat
        assert params.setdefault("lon",lon) == lon
        return await self._client.standard_request(method, url, *args, params=params, **kw)
    
    # Adds the Units
    _data_units = dict(
        temp = u.K,
        feels_like = u.K,
        pressure = u.hPa,
#        humidity = u.percent,
#        clouds = u.percent,
        dew_point = u.K,
        visibility = u.km,
        wind_speed = u.m/u.s,
        wind_gust = u.m/u.s,
        wind_deg = u.degree,
        rain = u.mm/u.hr,
        snow = u.mm/u.hr,
    )
    
    async def historic(self, ts:datetime.datetime, lang:str="en"):
        assert ts.tzinfo is not None, "Naive timestamps are not supported"
        dt = int(round(ts.timestamp()))
        ret = await self.get("onecall/timemachine",params=dict(dt=dt,lang=lang,units="standard"))
        ret.timezone_offset = datetime.timedelta(seconds = ret.timezone_offset)
        for d in ret.data:
            for k,units in self._data_units.items():
                v = getattr(d, k, None)
                if v is None:
                    continue
                setattr(d, k, u.Quantity(v, units))
            for k in "dt sunrise sunset".split():
                v = getattr(d, k, None)
                if v is None:
                    continue
                setattr(d, k, datetime.datetime.utcfromtimestamp(v).replace(tzinfo=datetime.timezone.utc))
        return ret

In [34]:
api_key_ow = """***REMOVED***""".strip()

In [35]:
coordinates = {
    "zurich": (47.38, 8.54),# "47.38°N 8.54°E",
}

In [36]:
async with OpenWeatherClient(
    api_key = api_key_ow,
) as client:
    zurich = client.station_at(*coordinates["zurich"])
    data = await zurich.historic(datetime.datetime.now(datetime.timezone.utc) - datetime.timedelta(hours=24))

data

namespace(lat=47.38,
          lon=8.54,
          timezone='Europe/Zurich',
          timezone_offset=datetime.timedelta(seconds=3600),
          data=[namespace(dt=datetime.datetime(2023, 3, 18, 17, 53, 28, tzinfo=datetime.timezone.utc),
                          sunrise=datetime.datetime(2023, 3, 18, 5, 33, 51, tzinfo=datetime.timezone.utc),
                          sunset=datetime.datetime(2023, 3, 18, 17, 34, 15, tzinfo=datetime.timezone.utc),
                          temp=287.38 <Unit('kelvin')>,
                          feels_like=286.27 <Unit('kelvin')>,
                          pressure=1013 <Unit('hectopascal')>,
                          humidity=54,
                          dew_point=278.21 <Unit('kelvin')>,
                          uvi=0,
                          clouds=75,
                          visibility=10000 <Unit('kilometer')>,
                          wind_speed=2.06 <Unit('meter / second')>,
                          wind_deg=320 <Unit('degree')>,
      